In [1]:
import pandas as pd
canaritos = 5
df = pd.read_csv('~/datasets/competencia_02_target.csv')
clientes = df[df["foto_mes"] == 202108]["numero_de_cliente"].unique()
df = df[df["numero_de_cliente"].isin(clientes)]

features_to_drop = ["cprestamos_prendarios", "mprestamos_prendarios", "cprestamos_personales", "mprestamos_personales", "clase_ternaria"]
df = df.drop(columns=features_to_drop)

KeyError: "['cprestamos_personales', 'mprestamos_personales'] not found in axis"

In [10]:
from dmeyf2025.processors.feature_processors import CleanZerosTransformer, DeltaLagTransformer, PercentileTransformer, PeriodStatsTransformer, TendencyTransformer, IntraMonthTransformer, RandomForestFeaturesTransformer, DatesTransformer, HistoricalFeaturesTransformer, AddCanaritos
from experiment_utils import *
def get_features(X, training_months):
    X_transformed = X
    
    X_transformed = apply_transformer(
        CleanZerosTransformer(),
        X_transformed,
        "CleanZerosTransformer",
        logger
    )
    X_transformed = apply_transformer(
        HistoricalFeaturesTransformer(),
        X_transformed,
        "HistoricalFeaturesTransformer",
        logger,
        parallel=True, parallelize_by='numero_de_cliente', n_jobs=-1
    )
    initial_cols =X_transformed.columns
    X_transformed = apply_transformer(
        PeriodStatsTransformer(
            periods=[6],
            exclude_cols=["foto_mes","numero_de_cliente","target","label","weight","clase_ternaria"]
        ),
        X_transformed,
        "PeriodStatsTransformer",
        logger,
        parallel=True, parallelize_by='numero_de_cliente', n_jobs=-1
    )
    new_cols = list(set(X_transformed.columns) - set(initial_cols))
    X_transformed = apply_transformer(
        TendencyTransformer(
            exclude_cols=["foto_mes","numero_de_cliente","target","label","weight","clase_ternaria"] + new_cols
        ),
        X_transformed,
        "TendencyTransformer",
        logger,
        parallel=True, parallelize_by='numero_de_cliente', n_jobs=-1
    )
    new_cols = list(set(X_transformed.columns) - set(initial_cols))
    X_transformed = apply_transformer(
        DeltaLagTransformer(
            n_lags=2,
            exclude_cols=["foto_mes","numero_de_cliente","target","label","weight","clase_ternaria"] + new_cols
        ),
        X_transformed,
        "DeltaLagTransformer",
        logger,
        parallel=True, parallelize_by='numero_de_cliente', n_jobs=-1
    )
    X_transformed = apply_transformer(
        PercentileTransformer(
            replace_original=True
        ),
        X_transformed,
        "PercentileTransformer",
        logger,
        parallel=True, parallelize_by='foto_mes', n_jobs=-1
    )
    return X_transformed

[LOG] FileHandler creado OK en: /home/martin232009/buckets/b1/experiment.log


In [14]:
features = get_features(df, [])
features = AddCanaritos(n_canaritos=5).fit_transform(features)

2025-11-17 21:58:39,327 - INFO - [CleanZerosTransformer] Iniciando…
2025-11-17 21:58:49,037 - INFO - [CleanZerosTransformer] Tiempo: 9.69s | Memoria antes: 4.966 GB | Memoria después: 4.966 GB | Diferencia: +0.000 GB | Shape: 4,414,000 filas × 150 columnas
2025-11-17 21:58:49,358 - INFO - [HistoricalFeaturesTransformer] Iniciando…
2025-11-17 22:00:57,314 - INFO - [HistoricalFeaturesTransformer] Tiempo: 127.95s | Memoria antes: 4.966 GB | Memoria después: 5.262 GB | Diferencia: +0.296 GB | Shape: 4,414,000 filas × 159 columnas
2025-11-17 22:00:57,749 - INFO - [PeriodStatsTransformer] Iniciando…
2025-11-17 22:02:47,536 - INFO - [PeriodStatsTransformer] Tiempo: 109.78s | Memoria antes: 5.262 GB | Memoria después: 11.445 GB | Diferencia: +6.183 GB | Shape: 4,414,000 filas × 347 columnas
2025-11-17 22:02:47,828 - INFO - [TendencyTransformer] Iniciando…
2025-11-17 22:16:08,145 - INFO - [TendencyTransformer] Tiempo: 800.30s | Memoria antes: 11.445 GB | Memoria después: 12.990 GB | Diferencia:

In [19]:
import pickle
import lightgbm as lgb

model_file = "/home/martin232009/buckets/b1/models/zlgbm-histfeatures_c5_gbzlgbm-histfeatures_s0.02_uFalse_537919.pkl"

In [20]:
with open(model_file, 'rb') as f:
    model = pickle.load(f)

In [21]:
features = features[features["foto_mes"] == 202108]
id = features["numero_de_cliente"]

probs = model.predict(features.drop(columns=["numero_de_cliente"]))
res = pd.DataFrame(id)
res["probs"] = probs
res.sort_values(by="probs", ascending=False)

,numero_de_cliente,probs
4607650,598771321,1.000000e+00
4606331,595282122,1.000000e+00
4702779,1248124471,1.000000e+00
4559624,279549429,1.000000e+00
4657809,880623336,1.000000e+00
...,...,...
4602399,579545920,7.352207e-13
4693186,1149612112,6.730547e-13
4575228,354004024,6.138994e-13
4624594,676876055,3.700955e-13


In [22]:
res.sort_values(by="probs", ascending=False)["numero_de_cliente"].head(12000).to_csv("entrega2.csv", index=False)